In [1]:
import numpy as np
import os
import time
from keras import backend as K
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
def save_model_weights(model,path):
    model.save_weights(path)
    return
def save_model_arch(model,path):
    model.save(path)

In [4]:
initial_model = applications.VGG19(weights="imagenet", include_top=True)

In [5]:
initial_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [6]:
x = Dense(2, activation='softmax',name='classifier')(initial_model.layers[-2].output)
tune_model = Model(initial_model.input, x)
opt = Adam(lr=0.001)
tune_model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#sparse_categorical_crossentropy

In [7]:
tune_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
path = 'data/'
target_size = (224, 224)
batch_size = 10
def get_batches(directory, target_size=target_size, batch_size=batch_size, shuffle=False):
 datagen = ImageDataGenerator()
 return datagen.flow_from_directory(directory=directory,
                                    target_size=target_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=shuffle)
batches = get_batches(path+'train', shuffle=True)
valid_batches = get_batches(path+'validation', shuffle=False)


Found 1242 images belonging to 2 classes.
Found 52 images belonging to 2 classes.


In [9]:
'''out=tune_model.fit_generator(batches, samples_per_epoch=batches.nb_sample,nb_epoch=3, 
                         validation_data=valid_batches,
                         nb_val_samples=valid_batches.nb_sample)'''
out=tune_model.fit_generator(batches, steps_per_epoch=batches.samples/batches.batch_size, epochs=30, validation_data=valid_batches, validation_steps=valid_batches.samples/valid_batches.batch_size)

Epoch 1/30
125/124 [==============================] - 1822s - loss: 7.9500 - acc: 0.5032 - val_loss: 7.7491 - val_acc: 0.5192
Epoch 2/30
125/124 [==============================] - 1822s - loss: 8.0462 - acc: 0.5008 - val_loss: 7.7491 - val_acc: 0.5192
Epoch 3/30
125/124 [==============================] - 1823s - loss: 7.9950 - acc: 0.5040 - val_loss: 7.7491 - val_acc: 0.5192
Epoch 4/30
125/124 [==============================] - 1831s - loss: 7.9950 - acc: 0.5040 - val_loss: 7.7491 - val_acc: 0.5192
Epoch 5/30
125/124 [==============================] - 1823s - loss: 8.0462 - acc: 0.5008 - val_loss: 7.7491 - val_acc: 0.5192
Epoch 6/30
125/124 [==============================] - 1825s - loss: 8.0462 - acc: 0.5008 - val_loss: 7.7491 - val_acc: 0.5192
Epoch 7/30
125/124 [==============================] - 1833s - loss: 8.0973 - acc: 0.4976 - val_loss: 7.7491 - val_acc: 0.5192
Epoch 8/30
125/124 [==============================] - 1829s - loss: 7.9950 - acc: 0.5040 - val_loss: 7.7491 - val_acc:

In [12]:
tune_model.save('tune_model_10eph.h5')